In [18]:
from validate_single_model import validate_pt_model_across_groups

import pandas as pd
import wandb
api = wandb.Api()
MODEL_ROOT = "/home/admindi/sbenites/WirelessLocation/data_processing/hyperparameter_tunning/CNN_hyperparameter_optimization"

groups = {
    "CNN_extra_features_no_leak_xy_free" : 
        {
            "model_storage": f"{MODEL_ROOT}/model_storage_extra_features_run_no_leak_xy_free",
            "database" : "wifi_fingerprinting_data_extra_features_no_leak"
        },
    "CNN_extra_features_no_leaking" : 
        {
            "model_storage": f"{MODEL_ROOT}/model_storage_extra_features_run_no_leaking",
            "database" : "wifi_fingerprinting_data_extra_features_no_leak"
        },
    "CNN_extra_features" : 
        {
            "model_storage": f"{MODEL_ROOT}/model_storage_extra_features_run2",
            "database" : "wifi_fingerprinting_data_extra_features"
        },
    "CNN__meters" : 
        {
            "model_storage": f"{MODEL_ROOT}/model_storage_meters",
            "database" : "wifi_fingerprinting_data_meters"
        },
    #"CNN_exponential" : 
    #    {
    #        "model_storage": f"{MODEL_ROOT}/model_storage_exponential",
    #        "database" : "wifi_fingerprinting_data_exponential"
    #    },
    "CNN_raw" : 
        {
            "model_storage":  f"{MODEL_ROOT}/model_storage_raw",
            "database" : "wifi_fingerprinting_data_raw"
        },
    "CNN_second_experiment" : 
        {
            "model_storage": f"{MODEL_ROOT}/model_storage_second",
            "database" : "wifi_fingerprinting_data"
        }
}

In [19]:


model_dict = {
    "CNN_extra_features_no_leak_xy_free" : [],
    "CNN_extra_features_no_leaking" : [],
    "CNN_extra_features" : [],
    "CNN__meters" : [],
    "CNN_exponential" : [],
    "CNN_raw" : [],
    "CNN_second_experiment" : [],
    "NN_second_experiment" : [],
    "CNN" : [],
    "NN" : [],
}
for group in groups.keys():
    runs = api.runs(
    "santi-mbenites-university-of-lisbon/wifi-rssi-gradient-search",
    filters={"group":group}
    )
    for run in runs:
        model_dict[group].append(run.name)

In [20]:
print(model_dict["CNN"])

[]


In [21]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import pickle
import traceback

def run_one(group, model):
    path = f"{groups[group]['model_storage']}/{model}.pt"
    database = groups[group]["database"]
    res = validate_pt_model_across_groups(model_path=path, db_name=database)
    return (res, path)

# Build all (group, model) pairs
pairs = [(g, m) for g in groups for m in model_dict[g]]

output_dict = {}
max_workers = min(32, max(1, len(pairs)))  # sensible cap

with ThreadPoolExecutor(max_workers=max_workers) as ex:
    future_to_meta = {
        ex.submit(run_one, g, m): (g, m)
        for g, m in pairs
    }
    with tqdm(total=len(future_to_meta), desc="Validating models", dynamic_ncols=True) as pbar:
        for fut in as_completed(future_to_meta):
            g, m = future_to_meta[fut]
            key = f"{g}_{m}"
            try:
                res, path = fut.result()
                output_dict[key] = (res, path)
                tqdm.write(f"Finished model {key}")
            except Exception as e:
                # Record the error so you can inspect later
                output_dict[key] = ("ERROR", f"{type(e).__name__}: {e}")
                tqdm.write(f"Failed model {key}: {type(e).__name__}: {e}")
                traceback.print_exc()
            finally:
                pbar.update(1)

with open('all_wandb_models_output.pkl', 'wb') as fp:
    pickle.dump(output_dict, fp)

print('Dictionary saved successfully to file')


Validating models:   0%|          | 0/78 [00:00<?, ?it/s]

Failed model CNN_extra_features_outdoor_indoor_and_garage_run0_depth9_model0: FileNotFoundError: [Errno 2] No such file or directory: '/home/admindi/sbenites/WirelessLocation/data_processing/hyperparameter_tunning/CNN_hyperparameter_optimization/model_storage_extra_features_run2/outdoor_indoor_and_garage_run0_depth9_model0.pt'
Failed model CNN_extra_features_outdoor_and_indoor_run0_depth0_model0_old: FileNotFoundError: [Errno 2] No such file or directory: '/home/admindi/sbenites/WirelessLocation/data_processing/hyperparameter_tunning/CNN_hyperparameter_optimization/model_storage_extra_features_run2/outdoor_and_indoor_run0_depth0_model0_old.pt'
Failed model CNN_extra_features_outdoor_indoor_and_garage_run1_depth7_model5: FileNotFoundError: [Errno 2] No such file or directory: '/home/admindi/sbenites/WirelessLocation/data_processing/hyperparameter_tunning/CNN_hyperparameter_optimization/model_storage_extra_features_run2/outdoor_indoor_and_garage_run1_depth7_model5.pt'
Failed model CNN_ex

Traceback (most recent call last):
  File "/tmp/ipykernel_514162/3034991761.py", line 28, in <module>
    res, path = fut.result()
                ~~~~~~~~~~^^
  File "/home/admindi/miniforge3/envs/location-env/lib/python3.13/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ~~~~~~~~~~~~~~~~~^^
  File "/home/admindi/miniforge3/envs/location-env/lib/python3.13/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/home/admindi/miniforge3/envs/location-env/lib/python3.13/concurrent/futures/thread.py", line 59, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/tmp/ipykernel_514162/3034991761.py", line 9, in run_one
    res = validate_pt_model_across_groups(model_path=path, db_name=database)
  File "/home/admindi/sbenites/WirelessLocation/validation/validate_single_model/validate_single_model.py", line 165, in validate_pt_model_across_groups
    model = _load_checkpointed_model(model_path)          

📡 indoor: 3 collections -> X(38036, 3), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 3), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 3), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 26), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 3), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 26), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 3), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 27), y(38036, 2)
📡 indoor: 3 collections -> X(38036, 3), y(38036, 2)

Traceback (most recent call last):
  File "/tmp/ipykernel_514162/3034991761.py", line 28, in <module>
    res, path = fut.result()
                ~~~~~~~~~~^^
  File "/home/admindi/miniforge3/envs/location-env/lib/python3.13/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ~~~~~~~~~~~~~~~~~^^
  File "/home/admindi/miniforge3/envs/location-env/lib/python3.13/concurrent/futures/_base.py", line 401, in __get_result
    raise self._exception
  File "/home/admindi/miniforge3/envs/location-env/lib/python3.13/concurrent/futures/thread.py", line 59, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/tmp/ipykernel_514162/3034991761.py", line 9, in run_one
    res = validate_pt_model_across_groups(model_path=path, db_name=database)
  File "/home/admindi/sbenites/WirelessLocation/validation/validate_single_model/validate_single_model.py", line 178, in validate_pt_model_across_groups
    y_pred = _batched_predict(model, Xg, batch_size=batch_

📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)

📡 garage: 5 collections -> X(105290, 3), y(105290, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 26), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 26), y(220184, 2)
📡 garage: 5 collections -> X(105290, 3), y(105290, 2)
📡 garage: 5 collections -> X(105290, 3), y(105290, 2)
📡 garage: 5 collections -> X(105290, 3), y(105290, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 outdoor: 11 collections -> X(220184, 27), y(220184, 2)
📡 garage: 5 collections -> X(105290, 3), y(

In [12]:
import pickle
import pandas as pd
with open('all_wandb_models_output.pkl', 'rb') as fp:
    models = pickle.load(fp)

from process_stored_data import build_df ,split_by_group, top_n_per_group 

df_all = build_df(models)
groups = split_by_group(df_all) 
best_by_group = top_n_per_group(groups, n=10, metric="overall_mae_dist", ascending=True)



In [ ]:
features= ["key","group",
           "garage_mae_dist","garage_rmse_dist","garage_n_samples",
           "indoor_mae_dist","indoor_rmse_dist","indoor_n_samples",
           "outdoor_mae_dist","outdoor_rmse_dist","outdoor_n_samples",
           "overall_mae_dist","overall_rmse_dist","overall_n_samples",
           ]

print(best_by_group["CNN_extra_features_no_leak_xy_free"].keys())
for group in groups.keys():
    #print(best_by_group[group][["key","overall_mae_dist","overall_rmse_dist","model_path"]].head()))
    dataframe = pd.DataFrame(best_by_group[group])
    print(f"Group:{group}")
    display(dataframe[features])

Index(['key', 'family', 'datasets', 'tag', 'run', 'depth', 'model_id',
       'db_name', 'model_path', 'garage_mae_dist', 'garage_mae_x',
       'garage_mae_y', 'garage_median_dist', 'garage_n_samples',
       'garage_p75_dist', 'garage_rmse_dist', 'garage_rmse_x', 'garage_rmse_y',
       'indoor_mae_dist', 'indoor_mae_x', 'indoor_mae_y', 'indoor_median_dist',
       'indoor_n_samples', 'indoor_p75_dist', 'indoor_rmse_dist',
       'indoor_rmse_x', 'indoor_rmse_y', 'outdoor_mae_dist', 'outdoor_mae_x',
       'outdoor_mae_y', 'outdoor_median_dist', 'outdoor_n_samples',
       'outdoor_p75_dist', 'outdoor_rmse_dist', 'outdoor_rmse_x',
       'outdoor_rmse_y', 'overall_mae_dist', 'overall_mae_x', 'overall_mae_y',
       'overall_median_dist', 'overall_n_samples', 'overall_p75_dist',
       'overall_rmse_dist', 'overall_rmse_x', 'overall_rmse_y', 'group'],
      dtype='object')
Group:CNN_extra_features_no_leak_xy_free


,key,group,garage_mae_dist,garage_rmse_dist,garage_n_samples,indoor_mae_dist,indoor_rmse_dist,indoor_n_samples,outdoor_mae_dist,outdoor_rmse_dist,outdoor_n_samples,overall_mae_dist,overall_rmse_dist,overall_n_samples
0,CNN_extra_features_no_leak_xy_free_outdoor_ind...,CNN_extra_features_no_leak_xy_free,0.134046,0.189865,103431,0.005131,0.008396,38036,0.157789,0.209565,220184,0.134943,0.182773,361651
1,CNN_extra_features_no_leak_xy_free_outdoor_ind...,CNN_extra_features_no_leak_xy_free,0.130669,0.186732,103431,0.003512,0.004899,38036,0.165276,0.215289,220184,0.138365,0.184995,361651
2,CNN_extra_features_no_leak_xy_free_outdoor_ind...,CNN_extra_features_no_leak_xy_free,0.133342,0.189066,103431,0.003944,0.004634,38036,0.166062,0.215434,220184,0.139654,0.185722,361651
3,CNN_extra_features_no_leak_xy_free_outdoor_ind...,CNN_extra_features_no_leak_xy_free,0.144092,0.197130,103431,0.005957,0.008966,38036,0.173425,0.221261,220184,0.147422,0.192032,361651
4,CNN_extra_features_no_leak_xy_free_outdoor_and...,CNN_extra_features_no_leak_xy_free,0.438479,0.484058,103431,0.002309,0.008509,38036,0.166690,0.216283,220184,0.227132,0.271013,361651
5,CNN_extra_features_no_leak_xy_free_outdoor_and...,CNN_extra_features_no_leak_xy_free,0.149107,0.201358,103431,0.742448,0.817431,38036,0.177462,0.224417,220184,0.228774,0.280191,361651
6,CNN_extra_features_no_leak_xy_free_outdoor_and...,CNN_extra_features_no_leak_xy_free,0.158154,0.207763,103431,0.722329,0.804685,38036,0.186590,0.231265,220184,0.234802,0.284852,361651
7,CNN_extra_features_no_leak_xy_free_outdoor_and...,CNN_extra_features_no_leak_xy_free,0.168017,0.216943,103431,0.721670,0.798158,38036,0.188812,0.233741,220184,0.238907,0.288298,361651
8,CNN_extra_features_no_leak_xy_free_outdoor_and...,CNN_extra_features_no_leak_xy_free,0.434067,0.478854,103431,0.005877,0.008849,38036,0.189264,0.234609,220184,0.239989,0.280718,361651
9,CNN_extra_features_no_leak_xy_free_outdoor_and...,CNN_extra_features_no_leak_xy_free,0.463319,0.512343,103431,0.009770,0.013535,38036,0.198230,0.241596,220184,0.254224,0.295043,361651


Index(['key', 'family', 'datasets', 'tag', 'run', 'depth', 'model_id',
       'db_name', 'model_path', 'garage_mae_dist', 'garage_mae_x',
       'garage_mae_y', 'garage_median_dist', 'garage_n_samples',
       'garage_p75_dist', 'garage_rmse_dist', 'garage_rmse_x', 'garage_rmse_y',
       'indoor_mae_dist', 'indoor_mae_x', 'indoor_mae_y', 'indoor_median_dist',
       'indoor_n_samples', 'indoor_p75_dist', 'indoor_rmse_dist',
       'indoor_rmse_x', 'indoor_rmse_y', 'outdoor_mae_dist', 'outdoor_mae_x',
       'outdoor_mae_y', 'outdoor_median_dist', 'outdoor_n_samples',
       'outdoor_p75_dist', 'outdoor_rmse_dist', 'outdoor_rmse_x',
       'outdoor_rmse_y', 'overall_mae_dist', 'overall_mae_x', 'overall_mae_y',
       'overall_median_dist', 'overall_n_samples', 'overall_p75_dist',
       'overall_rmse_dist', 'overall_rmse_x', 'overall_rmse_y', 'group'],
      dtype='object')
Group:CNN_extra_features_no_leaking


,key,group,garage_mae_dist,garage_rmse_dist,garage_n_samples,indoor_mae_dist,indoor_rmse_dist,indoor_n_samples,outdoor_mae_dist,outdoor_rmse_dist,outdoor_n_samples,overall_mae_dist,overall_rmse_dist,overall_n_samples
0,CNN_extra_features_no_leaking_outdoor_and_gara...,CNN_extra_features_no_leaking,0.422357,0.468634,103431,0.733766,0.825771,38036,0.459085,0.513862,220184,0.477470,0.533732,361651
1,CNN_extra_features_no_leaking_outdoor_indoor_a...,CNN_extra_features_no_leaking,0.441611,0.497439,103431,1.016957,1.071941,38036,0.549944,0.619798,220184,0.568078,0.632357,361651
2,CNN_extra_features_no_leaking_outdoor_and_gara...,CNN_extra_features_no_leaking,0.542350,0.595848,103431,1.078860,1.122678,38036,0.505221,0.558773,220184,0.576171,0.628684,361651
3,CNN_extra_features_no_leaking_outdoor_and_gara...,CNN_extra_features_no_leaking,0.726248,0.800504,103431,0.650381,0.711931,38036,0.646982,0.715044,220184,0.670009,0.739158,361651
4,CNN_extra_features_no_leaking_outdoor_indoor_a...,CNN_extra_features_no_leaking,1.141205,1.229041,103431,0.960831,1.014937,38036,1.086539,1.139442,220184,1.088952,1.151973,361651
5,CNN_extra_features_no_leaking_outdoor_indoor_a...,CNN_extra_features_no_leaking,1.270215,1.317149,103431,0.965177,0.986063,38036,1.392256,1.447642,220184,1.312435,1.361775,361651


Index(['key', 'family', 'datasets', 'tag', 'run', 'depth', 'model_id',
       'db_name', 'model_path', 'garage_mae_dist', 'garage_mae_x',
       'garage_mae_y', 'garage_median_dist', 'garage_n_samples',
       'garage_p75_dist', 'garage_rmse_dist', 'garage_rmse_x', 'garage_rmse_y',
       'indoor_mae_dist', 'indoor_mae_x', 'indoor_mae_y', 'indoor_median_dist',
       'indoor_n_samples', 'indoor_p75_dist', 'indoor_rmse_dist',
       'indoor_rmse_x', 'indoor_rmse_y', 'outdoor_mae_dist', 'outdoor_mae_x',
       'outdoor_mae_y', 'outdoor_median_dist', 'outdoor_n_samples',
       'outdoor_p75_dist', 'outdoor_rmse_dist', 'outdoor_rmse_x',
       'outdoor_rmse_y', 'overall_mae_dist', 'overall_mae_x', 'overall_mae_y',
       'overall_median_dist', 'overall_n_samples', 'overall_p75_dist',
       'overall_rmse_dist', 'overall_rmse_x', 'overall_rmse_y', 'group'],
      dtype='object')
Group:CNN_extra_features


...

,key,group,garage_mae_dist,garage_rmse_dist,garage_n_samples,indoor_mae_dist,indoor_rmse_dist,indoor_n_samples,outdoor_mae_dist,outdoor_rmse_dist,outdoor_n_samples,overall_mae_dist,overall_rmse_dist,overall_n_samples
0,CNN__meters_outdoor_only_run0_depth2_model4,CNN__meters,14.237996,15.706272,105290,17.028755,18.444482,38036,10.956097,12.604834,226610,12.514558,14.087975,369936


Index(['key', 'family', 'datasets', 'tag', 'run', 'depth', 'model_id',
       'db_name', 'model_path', 'garage_mae_dist', 'garage_mae_x',
       'garage_mae_y', 'garage_median_dist', 'garage_n_samples',
       'garage_p75_dist', 'garage_rmse_dist', 'garage_rmse_x', 'garage_rmse_y',
       'indoor_mae_dist', 'indoor_mae_x', 'indoor_mae_y', 'indoor_median_dist',
       'indoor_n_samples', 'indoor_p75_dist', 'indoor_rmse_dist',
       'indoor_rmse_x', 'indoor_rmse_y', 'outdoor_mae_dist', 'outdoor_mae_x',
       'outdoor_mae_y', 'outdoor_median_dist', 'outdoor_n_samples',
       'outdoor_p75_dist', 'outdoor_rmse_dist', 'outdoor_rmse_x',
       'outdoor_rmse_y', 'overall_mae_dist', 'overall_mae_x', 'overall_mae_y',
       'overall_median_dist', 'overall_n_samples', 'overall_p75_dist',
       'overall_rmse_dist', 'overall_rmse_x', 'overall_rmse_y', 'group'],
      dtype='object')
Group:CNN_raw


,key,group,garage_mae_dist,garage_rmse_dist,garage_n_samples,indoor_mae_dist,indoor_rmse_dist,indoor_n_samples,outdoor_mae_dist,outdoor_rmse_dist,outdoor_n_samples,overall_mae_dist,overall_rmse_dist,overall_n_samples
0,CNN_raw_outdoor_indoor_and_garage_run3_depth1_...,CNN_raw,2.848140,3.267849,105290,1.557744,1.984390,38036,2.743654,3.179710,226610,2.651460,3.081896,369936
1,CNN_raw_outdoor_and_indoor_run2_depth6_model3,CNN_raw,4.680081,5.109638,105290,1.182661,1.530274,38036,2.530180,2.981500,226610,3.003529,3.437992,369936
2,CNN_raw_outdoor_and_indoor_run1_depth8_model2,CNN_raw,4.649992,5.073529,105290,1.239062,1.573778,38036,2.575942,3.021094,226610,3.028796,3.456442,369936
3,CNN_raw_outdoor_and_garage_run2_depth1_model5,CNN_raw,2.875256,3.250333,105290,4.868566,5.242679,38036,2.821364,3.228601,226610,3.047191,3.441869,369936
4,CNN_raw_outdoor_and_garage_run0_depth4_model1,CNN_raw,2.857766,3.237640,105290,4.897172,5.290164,38036,2.832159,3.241021,226610,3.051767,3.450747,369936
5,CNN_raw_outdoor_and_indoor_run0_depth7_model3,CNN_raw,4.635305,5.056095,105290,1.385968,1.692608,38036,2.638159,3.066179,226610,3.077833,3.491315,369936
6,CNN_raw_outdoor_and_indoor_run3_depth8_model0,CNN_raw,4.609334,5.039682,105290,1.764755,2.022718,38036,2.767684,3.188984,226610,3.188730,3.595811,369936
7,CNN_raw_outdoor_and_garage_run3_depth7_model3,CNN_raw,8.065764,8.561891,105290,3.721694,4.011849,38036,8.102577,8.591376,226610,7.641667,8.112127,369936
8,CNN_raw_outdoor_and_garage_run1_depth8_model0,CNN_raw,8.237945,8.725018,105290,3.879451,4.165943,38036,8.275245,8.755125,226610,7.812663,8.274706,369936
9,CNN_raw_outdoor_indoor_and_garage_run0_depth9_...,CNN_raw,8.426630,8.905668,105290,4.050732,4.327592,38036,8.471522,8.938953,226610,8.004209,8.455350,369936


Index(['key', 'family', 'datasets', 'tag', 'run', 'depth', 'model_id',
       'db_name', 'model_path', 'garage_mae_dist', 'garage_mae_x',
       'garage_mae_y', 'garage_median_dist', 'garage_n_samples',
       'garage_p75_dist', 'garage_rmse_dist', 'garage_rmse_x', 'garage_rmse_y',
       'indoor_mae_dist', 'indoor_mae_x', 'indoor_mae_y', 'indoor_median_dist',
       'indoor_n_samples', 'indoor_p75_dist', 'indoor_rmse_dist',
       'indoor_rmse_x', 'indoor_rmse_y', 'outdoor_mae_dist', 'outdoor_mae_x',
       'outdoor_mae_y', 'outdoor_median_dist', 'outdoor_n_samples',
       'outdoor_p75_dist', 'outdoor_rmse_dist', 'outdoor_rmse_x',
       'outdoor_rmse_y', 'overall_mae_dist', 'overall_mae_x', 'overall_mae_y',
       'overall_median_dist', 'overall_n_samples', 'overall_p75_dist',
       'overall_rmse_dist', 'overall_rmse_x', 'overall_rmse_y', 'group'],
      dtype='object')
Group:CNN_second_experiment


,key,group,garage_mae_dist,garage_rmse_dist,garage_n_samples,indoor_mae_dist,indoor_rmse_dist,indoor_n_samples,outdoor_mae_dist,outdoor_rmse_dist,outdoor_n_samples,overall_mae_dist,overall_rmse_dist,overall_n_samples
0,CNN_second_experiment_all_data_run0_depth6_model9,CNN_second_experiment,0.389607,0.418051,105290,0.507899,0.555474,38036,0.391538,0.420695,226610,0.402952,0.433800,369936
1,CNN_second_experiment_outdoor_indoor_and_garag...,CNN_second_experiment,0.389493,0.417459,105290,0.504320,0.551425,38036,0.392351,0.421660,226610,0.403050,0.433807,369936
2,CNN_second_experiment_all_data_run3_depth5_model9,CNN_second_experiment,0.389690,0.417753,105290,0.504173,0.551249,38036,0.392317,0.421587,226610,0.403071,0.433827,369936
3,CNN_second_experiment_outdoor_indoor_and_garag...,CNN_second_experiment,0.389687,0.417747,105290,0.504144,0.551211,38036,0.392326,0.421598,226610,0.403072,0.433828,369936
4,CNN_second_experiment_outdoor_indoor_and_garag...,CNN_second_experiment,0.389671,0.417723,105290,0.504134,0.551199,38036,0.392335,0.421612,226610,0.403072,0.433829,369936
5,CNN_second_experiment_all_data_run1_depth2_model5,CNN_second_experiment,0.389685,0.417743,105290,0.504120,0.551181,38036,0.392332,0.421606,226610,0.403072,0.433829,369936
6,CNN_second_experiment_outdoor_indoor_and_garag...,CNN_second_experiment,0.389679,0.417734,105290,0.504110,0.551169,38036,0.392337,0.421613,226610,0.403073,0.433830,369936
7,CNN_second_experiment_all_data_run4_depth7_mod...,CNN_second_experiment,0.389664,0.417702,105290,0.503828,0.550815,38036,0.392410,0.421710,226610,0.403084,0.433844,369936
8,CNN_second_experiment_outdoor_only_run0_depth5...,CNN_second_experiment,0.392984,0.423412,105290,0.534843,0.589140,38036,0.385964,0.414937,226610,0.403270,0.435261,369936
9,CNN_second_experiment_outdoor_and_indoor_run4_...,CNN_second_experiment,0.391367,0.420169,105290,0.503190,0.550087,38036,0.392247,0.421386,226610,0.403404,0.434272,369936
